In [ ]:
# https://www.kaggle.com/c/bengaliai-cv19/discussion/123198
# 위와 같이 best single model에 들어가서 사람들이 어떤 식으로 모델링을 하고 점수를 어느정도 얻었는지 파악하고 
# 대회를 진행하면 비교적 수월하게 진행가능

In [ ]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import os

In [ ]:
data_dir = '../input/bengaliai-cv19'

#  Image visualization and folding

In [ ]:
files_train = [f'train_image_data_{fid}.parquet' for fid in range(4)]
# for문과 f string을 이용해서 불러온다.


### f string example ###
# name = 'Song' sex = 'male' 
# f'Hi, I am {name}. I am {sex}.'
# >>> 'Hi, I am song. I am male.'


In [ ]:
files_train

In [ ]:
train0 = pd.read_parquet('../input/bengaliai-cv19/train_image_data_0.parquet')

In [ ]:
train0.head()

In [ ]:
print(train0.shape)

# column이 많은 이유 : gray scale의 사진인데 (137 heights, 236 widths)크기의 이미지이므로 

print(137*236+1) 

# 여기서 +1은 'image_id'이다.

##### 결국 한 row가 하나의 이미지를 뜻한다. 즉 (137 multiply 236)를 일렬로 나열한 것이므로 이미지를 보고자 하면 다시 (137 multiply 236)형태로 만들어주면 되는 것.

In [ ]:
idx=0
img=train0.iloc[idx,1:].values.astype(np.uint8) # '1:'인 이유는 image_id 를 빼야하기 때문 

# 'astype(np.uint8)' : 굳이 큰 용량이 필요한 datatype을 사용할 필요가 없고 효율적으로 진행하기위해

In [ ]:
img.shape

In [ ]:
img.reshape(137,236).shape

In [ ]:
plt.imshow(img.reshape(137,236))

In [ ]:
plt.imshow(img.reshape(137,236), cmap = 'gray') # 흑백이미지 스케일로 보기

In [ ]:
len(train0) 

# 행의 수

In [ ]:
idx = np.random.randint(len(train0))

# 이미지 대회를 할때 중요한 것은 사진을 계속 보면서 어떻게 생겼는지 대충 파악을 해야 된다.
# 또한, 어떤 augmentation을 할지도 생각.
# 지금 이미지를 분석해보면 어디 쏠리지 않고 나름 중심을 두고 적혀있다. -> 사진의 quality가 괜찮다

img=train0.iloc[idx,1:].values.astype(np.uint8)

plt.imshow(255 - img.reshape(137,236), cmap = 'gray') # 흑백이미지 반전 스케일로 보기 like MNIST

# Multi-label stratification folding

In [ ]:
df_train = pd.read_csv('../input/bengaliai-cv19/train.csv')

In [ ]:
df_train.head()

In [ ]:
df_train.shape

### Checking the Distribution of label

In [ ]:
plt.figure(figsize=(10,50))
df_train['grapheme_root'].value_counts().sort_index().plot.barh()

# grapheme_root라는 label의 class가 총 168개(0~167)가 있는데
# 각 클래스가 어떤 count를 가지는지 보여주는 그래프

- Each class in grapheme_root is very unbalanced!
- In this condition, if we randomly sample, the distribution of classes for each fold may not be properly entered.
- If that happens, the model may not be able to train properly.
#### So we have to do a stratified fold

In [ ]:
plt.figure(figsize=(10,10))
df_train['vowel_diacritic'].value_counts().sort_index().plot.barh()

- Same situation as grapheme_root

In [ ]:
plt.figure(figsize=(10,7))
df_train['consonant_diacritic'].value_counts().sort_index().plot.barh()

- Same situation as grapheme_root

#### Now we have to use Stratified fold, but The stratified fold provided by sklearn is applied to only one label.
#### Now that we are dealing with three labels, we have to use 'iterative-stratification', a library that helps us fold while keeping the distribution of all three labels the same.

In [ ]:
!pip install iterative-stratification

from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

In [ ]:
df_train['id'] = df_train['image_id'].apply(lambda x: int(x.split('_')[1]))

In [ ]:
X = df_train[['id', 'grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values[:, 0] # id
y = df_train[['id', 'grapheme_root', 'vowel_diacritic', 'consonant_diacritic']].values[:, 1:] # target

In [ ]:
mskf = MultilabelStratifiedKFold(n_splits=6, random_state=1944, shuffle=True)

In [ ]:
df_train['fold'] = -1

In [ ]:
for i, (trn_idx, vid_idx) in enumerate(mskf.split(X,y)):
    df_train.loc[vid_idx, 'fold'] = i  

In [ ]:
df_train['fold'].value_counts()

In [ ]:
df_train.to_csv('df_folds.csv', index=False)

# Efficient learning process
- dataframe을 row by row로 잘라서 학습할 때 조금 더 빨리 할 수 있게 하는 방법
- 본 대회는 parquet파일 4개를 학습시켜서 합쳐야 하기 때문에 메모리가 크지 않으면 자칫 학습하다가 끊길수도 있다..
- pandas가 읽고 쓰는 속도가 생각보다 느린 편.
- 만약에 모델을 학습하는 코드를 만들었는데 pandas에서 불러오는 식으로 만들면 전체적인 학습 시간이 길어진다.
- So How to fix it??

In [ ]:
import joblib

from tqdm import tqdm
# tqdm : 반복문이 어디까지 진행되었는지 알고싶을때 사용

In [ ]:
train0.head()

In [ ]:
img_ids = train0['image_id'].values
img_array = train0.iloc[:, 1:].values

In [ ]:
for idx in tqdm(range(len(train0))):
    break

In [ ]:
img_id = img_ids[idx]
img = img_array[idx]

In [ ]:
os.mkdir('/kaggle/working/train_images/')

In [ ]:
joblib.dump(img, f'./train_images/{img_id}.pkl')

# pkl로 저장하면 아주 빨리 읽을 수 있다

In [ ]:
%%time
img0 = joblib.load(f'./train_images/{img_id}.pkl')

In [ ]:
%%time
train0.iloc[0, 1:].values

In [ ]:
25.2/1.8
# 14배 정도 시간차이가 난다.!!
# RAM이 작다면 pkl로 저장해서 load하는게 훨씬 효율적이다

In [ ]:
## Full code

# for fname in files_train: 
#     F = os.path.join(data_dir, fname)
#     train_image = pd.read_parquet(F)
#     img_ids = train_image['image_id'].values
#     img_array = train_image.iloc[:, 1:].values
#     for idx in tqdm(range(len(train_image))):
#         img_id = img_ids[idx]
#         img = img_array[idx]
#         joblib.dump(img, f'./train_images/{img_id}.pkl')


# Pytorch dataset

In [ ]:
import torch 

import warnings 
warnings.filterwarnings('ignore')

from torch.utils.data import Dataset
import matplotlib.pyplot as plt
import joblib

In [ ]:
df_train.head()

In [ ]:
index=0
HEIGHT=137
WIDTH=236

In [ ]:
img_ids = df_train['image_id'].values

In [ ]:
img_id = img_ids[index]

In [ ]:
img_id

In [ ]:
img = joblib.load(f'./train_images/{img_id}.pkl').astype(np.uint8)

In [ ]:
img.shape

In [ ]:
img = img.reshape(HEIGHT, WIDTH) 

In [ ]:
plt.imshow(img, cmap='gray')

In [ ]:
img = 255-img

In [ ]:
plt.imshow(img, cmap='gray')

In [ ]:
# pytorch model에 넣어주려면 channel이 있어야 함

print(img[:, :, np.newaxis].shape)
img = img[:, :, np.newaxis]

In [ ]:
label_1 = df_train.iloc[index].grapheme_root
label_2 = df_train.iloc[index].vowel_diacritic
label_3 = df_train.iloc[index].consonant_diacritic

- 특정 index에 맞는 image pkl을 불러온다
- 그 pkl을 reshape 한다
- channel 추가(for pytorch)
- 그 image가 가지고 있는 각 label별 클래스를 다 가져온다

In [ ]:
class BengaliDataset(Dataset):
    def __init__(self, csv, img_height, img_width):
        self.csv = csv.reset_index()
        self.img_ids = csv['image_id'].values
        self.img_height = img_height
        self.img_width = img_width
    
    def __len__(self):
        return len(self.csv) # Dataset 길이 ; 지금은 csv파일이 dataset이므로 csv파일 길이를 return하면 됨
    
    def __getitem__(self, index):
        img_id = self.img_ids[index]
        img = joblib.load(f'./train_images/{img_id}.pkl')
        img = img.reshape(self.img_height, self.img_width).astype(np.uint8)
        img = 255-img
        img = img[:,:,np.newaxis]
        
        label_1 = self.csv.iloc[index].grapheme_root
        label_2 = self.csv.iloc[index].vowel_diacritic
        label_3 = self.csv.iloc[index].consonant_diacritic
        
        return (torch.tensor(img, dtype=torch.float).permute(2,0,1), torch.tensor(label_1, dtype=torch.long), 
                torch.tensor(label_2, dtype=torch.long), torch.tensor(label_3, dtype=torch.long))


##### For Pytorch model

- (B, W, H, C) -> (B, C, W, H)
- B : Batch, W : Width, H : Height, C : Channel
- (16, 137, 236, 1) -> (16, 1, 137, 236)

In [ ]:
print(img.shape)
print(torch.tensor(img).shape)
print(torch.tensor(img).permute(2,0,1).shape) # permute : 차원을 바꿔주기 위해 사용

In [ ]:
df_train['fold'] = pd.read_csv('./df_folds.csv')['fold']

In [ ]:
df_train['fold']

In [ ]:
trn_fold = [i for i in range(6) if i not in [5]]

In [ ]:
trn_fold

In [ ]:
vid_fold = [5]

In [ ]:
trn_idx = df_train.loc[df_train['fold'].isin(trn_fold)].index
vid_idx = df_train.loc[df_train['fold'].isin(vid_fold)].index

In [ ]:
trn_dataset = BengaliDataset(csv=df_train.loc[trn_idx], img_height = HEIGHT, img_width = WIDTH)

In [ ]:
trn_dataset[0][0].shape

In [ ]:
plt.imshow(trn_dataset[0][0].permute(1,2,0).numpy()[:,:,0], cmap='gray')

In [ ]:
print(trn_dataset[0][0].permute(1,2,0).numpy().shape) # channel을 없애주어야 사진을 볼 수 있다.
print(trn_dataset[0][0].permute(1,2,0).numpy()[:,:,0].shape) # '[:,:,0]'을 통해 channel 삭제

In [ ]:
# idx = 0
# idx += 1
# plt.imshow(trn_dataset[idx][0].permute(1,2,0).numpy()[:,:,0], cmap='gray')